# Fiddler Programmatic Dashboards Script

Export segments, custom metrics, and charts from one Fiddler model and import them to another model, with support for:
* Cross-instance transfers (different Fiddler instances)
* Cross-model transfers (different models on same or different instances)
* Automatic schema validation with error handling for incompatible columns

## Prerequisites

* Source Fiddler instance URL and API token
* Target Fiddler instance URL and API token (can be same as source)
* Source and target models must exist
* `fiddler_utils` package installed (run cell below)

In [1]:
# Install dependencies
# %pip install -q fiddler-client

# Install fiddler_utils from parent directory
# If not already installed, run from repo root: pip install -e .
import sys

sys.path.insert(0, "..")

In [2]:
import fiddler as fdl
from fiddler_utils import (
    ConnectionManager,
    SegmentManager,
    CustomMetricManager,
    SchemaValidator,
)
from requests import HTTPError, Response
from fiddler.libs.http_client import RequestClient

print(f"Fiddler client version: {fdl.__version__}")
print("fiddler_utils: Successfully imported")

Fiddler client version: 3.10.0
fiddler_utils: Successfully imported


## Configuration

In [3]:
# Source Fiddler Instance
SOURCE_URL = "https://tech-exercise.cloud.fiddler.ai"  # e.g., 'https://source.fiddler.ai'
SOURCE_TOKEN = "4hsiemgkUi4i2TI18THfFxlctAIrH2yDGxEHpBfBXpk"
SOURCE_PROJECT_NAME = "dashboard_templates_001"
SOURCE_MODEL_NAME = "fiddler_rag_llm_chatbot"
SOURCE_MODEL_VERSION = ""  # Optional, leave empty for unversioned models

# Target Fiddler Instance (can be same as source)
TARGET_URL = "https://tech-exercise.cloud.fiddler.ai"  # e.g., 'https://target.fiddler.ai'
TARGET_TOKEN = "4hsiemgkUi4i2TI18THfFxlctAIrH2yDGxEHpBfBXpk"
TARGET_PROJECT_NAME = "my_test_use_case_009"
TARGET_MODEL_NAME = "fiddler_rag_llm_chatbot"
TARGET_MODEL_VERSION = ""  # Optional, leave empty for unversioned models

# Asset Selection (empty lists = export all)
SEGMENTS_TO_EXPORT = []  # e.g., ['segment1', 'segment2'] or [] for all
CUSTOM_METRICS_TO_EXPORT = []  # e.g., ['metric1'] or [] for all


# Chart Export Options (for bonus section)
SOURCE_DASHBOARD_ID = "b2920aa0-b073-44f6-ac30-17d89c1ae703"  # Dashboard UUID to export charts from (leave empty to skip)
CHART_IDS_TO_EXPORT = []  # e.g., ['uuid1', 'uuid2'] or [] for none

## Setup Connections and Initialize Managers

In [4]:
# Setup connection manager for handling multiple instances
conn_mgr = ConnectionManager(log_level="WARNING")
conn_mgr.add("source", url=SOURCE_URL, token=SOURCE_TOKEN)
conn_mgr.add("target", url=TARGET_URL, token=TARGET_TOKEN)

# Initialize asset managers
segment_mgr = SegmentManager()
metric_mgr = CustomMetricManager()

print("✓ Connection manager configured")
print("✓ Asset managers initialized")

✓ Connection manager configured
✓ Asset managers initialized


## Fetch Source and Target Models

In [5]:
# Connect to source and get model
with conn_mgr.use("source"):
    source_project = fdl.Project.from_name(SOURCE_PROJECT_NAME)
    source_model_kwargs = {"project_id": source_project.id, "name": SOURCE_MODEL_NAME}
    if SOURCE_MODEL_VERSION:
        source_model_kwargs["version"] = SOURCE_MODEL_VERSION

    source_model = fdl.Model.from_name(**source_model_kwargs)
    print(f"Source model: {source_model.name} (ID: {source_model.id})")

# Connect to target and get model
with conn_mgr.use("target"):
    target_project = fdl.Project.from_name(TARGET_PROJECT_NAME)
    target_model_kwargs = {"project_id": target_project.id, "name": TARGET_MODEL_NAME}
    if TARGET_MODEL_VERSION:
        target_model_kwargs["version"] = TARGET_MODEL_VERSION

    target_model = fdl.Model.from_name(**target_model_kwargs)
    print(f"Target model: {target_model.name} (ID: {target_model.id})")

Source model: fiddler_rag_llm_chatbot (ID: 2cc87e3b-d94e-4e4f-9853-59c43755dc95)
Target model: fiddler_rag_llm_chatbot (ID: 999330d6-3220-4616-b86c-044b1495ba61)


## Schema Comparison

Compare source and target model schemas to identify potential compatibility issues.

In [6]:
# Get schema information from both models
with conn_mgr.use("source"):
    source_columns = SchemaValidator.get_model_columns(source_model)
    print(f"Source model has {len(source_columns)} columns")

with conn_mgr.use("target"):
    target_columns = SchemaValidator.get_model_columns(target_model)
    print(f"Target model has {len(target_columns)} columns")

with conn_mgr.use("source"):
    comparison = SchemaValidator.compare_schemas(source_model, target_model)

print("\n" + "=" * 60)
print("SCHEMA COMPARISON")
print("=" * 60)
print(f"\nCommon columns: {len(comparison.in_both)}")

if comparison.only_in_source:
    print(
        f"\n⚠️  Columns in SOURCE but MISSING in TARGET ({len(comparison.only_in_source)}):"
    )
    for col in sorted(comparison.only_in_source):
        col_info = source_columns[col]
        print(f"  - {col} ({col_info.role}, {col_info.data_type})")
else:
    print("\n✅ No missing columns - all source columns exist in target")

if comparison.only_in_target:
    print(
        f"\n📋 Columns in TARGET but NOT in SOURCE ({len(comparison.only_in_target)}):"
    )
    for col in sorted(list(comparison.only_in_target)[:5]):
        col_info = target_columns[col]
        print(f"  - {col} ({col_info.role}, {col_info.data_type})")
    if len(comparison.only_in_target) > 5:
        print(f"  ... and {len(comparison.only_in_target) - 5} more")

if comparison.type_mismatches:
    print(f"\n⚠️  Data type DIFFERENCES ({len(comparison.type_mismatches)}):")
    for col, (source_type, target_type) in list(comparison.type_mismatches.items())[:5]:
        print(f"  - {col}: source={source_type} → target={target_type}")
else:
    print("\n✅ All common columns have matching data types")

print(
    f"\n{'✅' if comparison.is_compatible else '⚠️ '} Schema compatibility: {comparison.is_compatible}"
)
print("=" * 60)

Source model has 21 columns
Target model has 21 columns

SCHEMA COMPARISON

Common columns: 21

✅ No missing columns - all source columns exist in target

✅ All common columns have matching data types

✅ Schema compatibility: True


## Export Segments

Export segments from source model using `SegmentManager`.

In [7]:
print("\n" + "=" * 60)
print("EXPORTING SEGMENTS")
print("=" * 60)

with conn_mgr.use("source"):
    # Export segments (filtered by name if specified)
    exported_segments = segment_mgr.export_assets(
        model_id=source_model.id,
        names=SEGMENTS_TO_EXPORT if SEGMENTS_TO_EXPORT else None,
    )

    print(f"\n✓ Exported {len(exported_segments)} segment(s)")

    for seg_data in exported_segments:
        print(f"\n  Segment: {seg_data.name}")
        print(f"    Definition: {seg_data.data['definition']}")
        print(f"    Referenced columns: {seg_data.referenced_columns}")


EXPORTING SEGMENTS

✓ Exported 1 segment(s)

  Segment: Non-null Responses
    Definition: is_not_null("response")
    Referenced columns: {'response'}


## Import Segments

Import segments to target model with automatic validation.

In [8]:
print("\n" + "=" * 60)
print("IMPORTING SEGMENTS")
print("=" * 60)

with conn_mgr.use("target"):
    # Import with validation and error handling
    segment_result = segment_mgr.import_assets(
        target_model_id=target_model.id,
        assets=exported_segments,
        validate=True,
        dry_run=False,
        skip_invalid=True,
        overwrite=False,
    )

    print("\nResults:")
    print(f"  ✅ Successfully imported: {segment_result.successful}")
    print(f"  🔄 Skipped (existing): {segment_result.skipped_existing}")
    print(f"  ⊘  Skipped (invalid): {segment_result.skipped_invalid}")
    print(f"  ❌ Failed: {segment_result.failed}")

    if segment_result.errors:
        print("\n  Errors:")
        for name, error in segment_result.errors:
            print(f"    - {name}: {error}")


IMPORTING SEGMENTS

Results:
  ✅ Successfully imported: 1
  🔄 Skipped (existing): 0
  ⊘  Skipped (invalid): 0
  ❌ Failed: 0


## Export Custom Metrics

Export custom metrics from source model using `CustomMetricManager`.

In [9]:
print("\n" + "=" * 60)
print("EXPORTING CUSTOM METRICS")
print("=" * 60)

with conn_mgr.use("source"):
    # Export custom metrics (filtered by name if specified)
    exported_metrics = metric_mgr.export_assets(
        model_id=source_model.id,
        names=CUSTOM_METRICS_TO_EXPORT if CUSTOM_METRICS_TO_EXPORT else None,
    )

    print(f"\n✓ Exported {len(exported_metrics)} custom metric(s)")

    for metric_data in exported_metrics:
        print(f"\n  Metric: {metric_data.name}")
        print(f"    Definition: {metric_data.data['definition']}")
        print(f"    Referenced columns: {metric_data.referenced_columns}")

        # Show complexity info
        metadata = metric_data.data["metadata"]
        metric_type = "Aggregation" if metadata["is_aggregation"] else "Simple"
        print(f"    Type: {metric_type}")
        if metadata["functions_used"]:
            print(f"    Functions: {', '.join(metadata['functions_used'])}")


EXPORTING CUSTOM METRICS

✓ Exported 1 custom metric(s)

  Metric: Total Non-Null Responses
    Definition: sum(if(is_not_null("response"), 1, 0))
    Referenced columns: {'response'}
    Type: Aggregation
    Functions: is_not_null, if, sum


## Import Custom Metrics

Import custom metrics to target model with automatic validation.

In [10]:
print("\n" + "=" * 60)
print("IMPORTING CUSTOM METRICS")
print("=" * 60)

with conn_mgr.use("target"):
    # Import with validation and error handling
    metric_result = metric_mgr.import_assets(
        target_model_id=target_model.id,
        assets=exported_metrics,
        validate=True,
        dry_run=False,
        skip_invalid=True,
        overwrite=False,
    )

    print("\nResults:")
    print(f"  ✅ Successfully imported: {metric_result.successful}")
    print(f"  🔄 Skipped (existing): {metric_result.skipped_existing}")
    print(f"  ⊘  Skipped (invalid): {metric_result.skipped_invalid}")
    print(f"  ❌ Failed: {metric_result.failed}")

    if metric_result.errors:
        print("\n  Errors:")
        for name, error in metric_result.errors:
            print(f"    - {name}: {error}")


IMPORTING CUSTOM METRICS

Results:
  ✅ Successfully imported: 1
  🔄 Skipped (existing): 0
  ⊘  Skipped (invalid): 0
  ❌ Failed: 0


## Summary Report

In [11]:
print("\n" + "=" * 60)
print("EXPORT/IMPORT SUMMARY")
print("=" * 60)

print("\nSegments:")
print(f"  Total exported: {len(exported_segments)}")
print(f"  Successfully imported: {segment_result.successful}")
print(f"  Skipped (existing): {segment_result.skipped_existing}")
print(f"  Skipped (invalid): {segment_result.skipped_invalid}")
print(f"  Failed: {segment_result.failed}")

print("\nCustom Metrics:")
print(f"  Total exported: {len(exported_metrics)}")
print(f"  Successfully imported: {metric_result.successful}")
print(f"  Skipped (existing): {metric_result.skipped_existing}")
print(f"  Skipped (invalid): {metric_result.skipped_invalid}")
print(f"  Failed: {metric_result.failed}")

total_success = segment_result.successful + metric_result.successful
total_skipped_existing = (
    segment_result.skipped_existing + metric_result.skipped_existing
)
total_skipped_invalid = segment_result.skipped_invalid + metric_result.skipped_invalid
total_failed = segment_result.failed + metric_result.failed
total_exported = len(exported_segments) + len(exported_metrics)

print("\n" + "=" * 60)
print(f"OVERALL: {total_success}/{total_exported} assets successfully imported")
if total_skipped_existing > 0:
    print(f"  {total_skipped_existing} skipped (already exist on target)")
if total_skipped_invalid > 0:
    print(f"  {total_skipped_invalid} skipped (validation errors)")
if total_failed > 0:
    print(f"  {total_failed} failed during import")
print("=" * 60)


EXPORT/IMPORT SUMMARY

Segments:
  Total exported: 1
  Successfully imported: 1
  Skipped (existing): 0
  Skipped (invalid): 0
  Failed: 0

Custom Metrics:
  Total exported: 1
  Successfully imported: 1
  Skipped (existing): 0
  Skipped (invalid): 0
  Failed: 0

OVERALL: 2/2 assets successfully imported


## Export/Import Charts

Demonstrate cross-instance chart transfer using `ChartManager`.

This section shows how fiddler_utils simplifies chart transfers.

**To use this section:**
1. Set `SOURCE_DASHBOARD_ID` in the configuration cell above (find dashboard ID in Fiddler UI URL)
2. OR set `CHART_IDS_TO_EXPORT` to manually specify chart UUIDs
3. Run the cells below to export and import charts

**Note:** Chart API uses unofficial Fiddler endpoints and may change without notice.

In [12]:
from fiddler_utils import ChartManager

print("\n" + "=" * 60)
print("INITIALIZING CHART MANAGERS")
print("=" * 60)

# Create separate managers for source and target
# Each needs its own URL/token for RequestClient
source_chart_mgr = ChartManager(url=SOURCE_URL, token=SOURCE_TOKEN)
target_chart_mgr = ChartManager(url=TARGET_URL, token=TARGET_TOKEN)

print("\n✓ Source ChartManager initialized")
print("✓ Target ChartManager initialized")


INITIALIZING CHART MANAGERS

✓ Source ChartManager initialized
✓ Target ChartManager initialized


In [13]:
print("\n" + "=" * 60)
print("EXPORTING CHARTS")
print("=" * 60)

# Determine export method
if SOURCE_DASHBOARD_ID:
    print(f"\nExporting charts from dashboard: {SOURCE_DASHBOARD_ID}")
elif CHART_IDS_TO_EXPORT:
    print(f"\nExporting {len(CHART_IDS_TO_EXPORT)} charts by ID")
else:
    print("\n⚠️  No SOURCE_DASHBOARD_ID or CHART_IDS_TO_EXPORT specified.")
    print("   Set one of these in the configuration cell to export charts.")
    exported_charts = []

if SOURCE_DASHBOARD_ID or CHART_IDS_TO_EXPORT:
    with conn_mgr.use("source"):
        try:
            # Export charts using dashboard_id or chart_ids
            exported_charts = source_chart_mgr.export_charts(
                dashboard_id=SOURCE_DASHBOARD_ID if SOURCE_DASHBOARD_ID else None,
                chart_ids=CHART_IDS_TO_EXPORT if CHART_IDS_TO_EXPORT else None,
            )

            print(f"\n✓ Exported {len(exported_charts)} chart(s)\n")

            # Display exported chart details
            for i, chart in enumerate(exported_charts, 1):
                print(f"{i}. {chart.get('title', 'Untitled')}")
                print(f"   Type: {chart.get('query_type', 'unknown')}")

                # Show data source info
                data_source = chart.get("data_source", {})
                queries = data_source.get("queries", [])

                if queries:
                    # Show first query details for monitoring charts
                    query = queries[0]
                    metric_info = []

                    if "baseline_name" in query:
                        metric_info.append(f"baseline={query['baseline_name']}")
                    if query.get("metric_type") == "custom":
                        metric_info.append(
                            f"custom_metric={query.get('metric', 'N/A')}"
                        )
                    if "segment" in query and query["segment"]:
                        metric_info.append(f"segment={query['segment']}")

                    if metric_info:
                        print(f"   Dependencies: {', '.join(metric_info)}")
                print()
        except Exception as e:
            print(f"\n❌ Failed to export charts: {e}")
            exported_charts = []


EXPORTING CHARTS

Exporting charts from dashboard: b2920aa0-b073-44f6-ac30-17d89c1ae703

✓ Exported 6 chart(s)

1. Embedding fiddler_rag_llm_chatbot (v1)
   Type: EMBEDDING

2. Traffic fiddler_rag_llm_chatbot (v1)
   Type: MONITORING
   Dependencies: custom_metric=47abbab9-398d-469a-84ba-106fcbfbb5c2, segment={'id': 'c845459f-9c83-4dbd-9036-f0d78a5ca308'}

3. DI Violations fiddler_rag_llm_chatbot (v1)
   Type: MONITORING

4. Fast Trust Models Faithfulness Frequency Chart fiddler_rag_llm_chatbot (v1)
   Type: MONITORING

5. PII Frequency Chart fiddler_rag_llm_chatbot (v1)
   Type: MONITORING

6. Sentiment Frequency Chart fiddler_rag_llm_chatbot (v1)
   Type: MONITORING



In [14]:
if not exported_charts:
    print("\n⊘ No charts to import. Skipping import.")
    chart_result = {"successful": 0, "failed": 0, "errors": []}
else:
    print("\n" + "=" * 60)
    print("IMPORTING CHARTS")
    print("=" * 60)

    with conn_mgr.use("target"):
        # Perform actual import
        chart_result = target_chart_mgr.import_charts(
            target_project_id=target_project.id,
            target_model_id=target_model.id,
            charts=exported_charts,
            validate=True,
            dry_run=False,  # Actually create charts
        )

        print("\nResults:")
        print(f"  ✅ Successfully imported: {chart_result['successful']}")
        print(f"  ❌ Failed: {chart_result['failed']}")

        if chart_result.get("errors"):
            print(f"\n  Errors encountered:")
            for title, error in chart_result["errors"]:
                print(f"    • {title}")
                print(f"      {error}")

        # Update overall summary
        if chart_result["successful"] > 0:
            print(
                f"\n✓ Successfully imported {chart_result['successful']} chart(s) to target model"
            )


IMPORTING CHARTS

Results:
  ✅ Successfully imported: 6
  ❌ Failed: 0

✓ Successfully imported 6 chart(s) to target model


## Package Charts Into a New Default Dashboard

In [16]:
client = RequestClient(
    TARGET_URL,
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {TARGET_TOKEN}",
    },
)

def create_dashboard(
    project_name: str,
    model_name: str,
    charts: list,
    dashboard: dict,
) -> dict:
    dashboards_url = "v2/dashboards"

    dashboard["organization_name"] = fdl.conn.organization_name
    dashboard["project_name"] = project_name

    try:
        # Map the chart titles to uuids
        chart_titles = {chart["title"]: chart["id"] for chart in charts}
        # Get model name from yaml config
        model_name = dashboard.get("model_name", None)
        dashboard.pop("model_name", None)

        # Replace chart titles with uuids in dashboard yaml
        for index, saved_chart in enumerate(dashboard.get("layouts")):
            chart_title = saved_chart.get("chart_title")
            saved_chart["chart_uuid"] = chart_titles.get(chart_title)
            saved_chart.pop("chart_title", None)

        dashboard_resp: Response = client.post(url=dashboards_url, data=dashboard)
        
        project = fdl.Project.get_or_create(name=project_name)
        model = fdl.Model.from_name(name=model_name, project_id=project.id)

        # Set the created dashboard as default
        default_dashboard_url = f"v3/models/{model.id}/default-dashboard"

        payload = {}
        payload["dashboard_uuid"] = dashboard_resp.json()["data"].get("uuid")

        client.put(url=default_dashboard_url, data=payload)

        return dashboard_resp

    except HTTPError as hex:
        print(
            f"HTTPError occured: {hex.response.text} with error code {hex.response.status_code}"
        )
        raise hex


def generate_chart_layout(chart_title, x, y):
    return {
        "chart_title": chart_title,
        "grid_props": {
            "height": 1,
            "position_x": x,
            "position_y": y,
            "width": 1,
        },
    }

def generate_chart_layouts(chart_titles):
    chart_layouts = []
    
    for i in range(len(chart_titles)):
        chart_title = chart_titles[i]
        x = i % 2
        y = i // 2
        chart_layouts.append(generate_chart_layout(chart_title, x, y))

    return chart_layouts


charts_url = f"/v3/charts?filter=%7B%22condition%22:%22AND%22,%22rules%22:[%7B%22field%22:%22project_name%22,%22operator%22:%22equal%22,%22value%22:%22{TARGET_PROJECT_NAME}%22%7D]%7D&search=&offset=0&limit=40"
charts_response = client.get(url=charts_url)
charts = charts_response.json().get("data").get("items")


# List out chart titles (out of order)
chart_titles = {chart["title"]: chart["id"] for chart in charts}


# Dashboards definition with chart order (YAML version below)
dashboard = {
    "layouts": generate_chart_layouts(list(chart_titles.keys())),
    "model_name": TARGET_MODEL_NAME,
    "options": {
        "filters": {"time_label": "6m", "time_zone": "America/Los_Angeles"}
    },
    "organization_name": fdl.conn.organization_name,
    "project_name": TARGET_PROJECT_NAME,
    "title": "test_dashboard",
}

# Run command to create dashboard(s) according to order in the yaml definition
dashboard_response = create_dashboard(
    project_name=TARGET_PROJECT_NAME, model_name=TARGET_MODEL_NAME, charts=charts, dashboard=dashboard
)

project = fdl.Project.from_name(TARGET_PROJECT_NAME)
model = fdl.Model.from_name(name=TARGET_MODEL_NAME, project_id=project.id)

# # Set the created dashboard as default
default_dashboard_url = f"v3/models/{model.id}/default-dashboard"

payload = {}
payload["dashboard_uuid"] = dashboard_response.json()['data']['uuid']

default_dashboard_response = client.put(url=default_dashboard_url, data=payload)

if default_dashboard_response.status_code == 200:
    print(f"  ✅ Successfully created new default dashboard")

  ✅ Successfully created new default dashboard
